In [92]:
#!python -m pip install tensorflow

In [23]:
from IPython.display import display, Image
import cv2 as cv
from PIL import Image

#august
import os
import pandas as pd
import numpy as np


#!pip install tensorflow
import tensorflow
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder

In [3]:
#display(Image(filename='brad.jpg'))
#display(Image(filename='kim.jpg'))
face_cascade = cv.CascadeClassifier("haarcascade_frontalface_default.xml")
eye_cascade = cv.CascadeClassifier("haarcascade_eye.xml")

In [4]:
def coords_for_crop(img_address):

    img = cv.imread(img_address)
    im = Image.open(img_address) 

    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray)

    xUpper, yUpper, xLower, yLower = 0,0,0,0
    
    

    for (x, y, w, h) in faces:
        #print(w,h,type(w),type(h))
        if (w>600 and h> 600):
            return x,y,x+w,y+h
        
        
        #cv.rectangle(img, (x, y), (x+w, y+h), (0, 0, 255), 2)
        #roi_gray = gray[y : y + h, x : x + h]
        #roi_color = img[y : y + h, x : x + h]
        # Otsi silmi vaid leitud nägude aladelt (kollased ristkülikud)
        #eyes = eye_cascade.detectMultiScale(roi_gray)
        #for (ex, ey, ew, eh) in eyes:
            #cv.rectangle(roi_color, (ex, ey), (ex+ew, ey+eh), (0, 255, 255), 2)


    # Otsi silmi kogu fotolt (rohelised ristkülikud)
    #eyes = eye_cascade.detectMultiScale(gray)
    #for (ex, ey, ew, eh) in eyes:
        #cv.rectangle(img, (ex, ey), (ex+ew, ey+eh), (0, 255, 0), 2)

    #im1 = im.crop((xUpper, yUpper, xLower, yLower))
    #im1.show()
    #cv.waitKey(0)
    #cv.destroyAllWindows()

In [27]:
# Pildi salvestamiseks
#cv.imwrite('jott_koopia.png', img)

In [17]:
import os
import pandas as pd
from PIL import Image
from numpy import asarray

# Defineerime CSV faili tee ja piltide peamise kausta tee
csv_file_path = './ratings.csv'
images_folder_path = './people'

# Laadime CSV faili
df = pd.read_csv(csv_file_path, header=None)

# Loome tühja DataFrame, kuhu salvestame ümberstruktureeritud andmed
columns = ['usaldusväärsus', 'ilu', 'intelligents', "pilt"]
andmestik_df = pd.DataFrame(columns=columns)

# Itereerime läbi CSV andmete ja lisame need ümberstruktureeritult uude DataFrame'i
for index, row in df.iterrows():
    # Määrame kausta nime vastavalt rea indeksile
    folder_name = f'id{index + 1}'
    folder_path = os.path.join(images_folder_path, folder_name)
    
    # Lisame andmed vastavalt kaustas olevatele piltidele
    for i in range(0, len(row)-3, 3):
        image_name = f'Allalaaditud fail ({i//3 + 1}).jpeg'  # Oletame, et pildid on nimedega img1.jpg, img2.jpg, img3.jpg
        image_path = os.path.join(folder_path, image_name)
        
        # load the image and convert into
        # numpy array
        img = Image.open(image_path)
        
        #crop image so it will only contain the face of the person
        img = img.crop(coords_for_crop(image_path))
        
        #resize image to 600x600 
        img = img.resize((600,600))
        
        ##
        #img.show()
        ##
        
        # asarray() class is used to convert
        # PIL images into NumPy arrays
        image_array = asarray(img)
        
        
        image_data = pd.DataFrame({
            'usaldusväärsus': [row[i]],
            'ilu': [row[i+1]],
            'intelligents': [row[i+2]],
            'pilt': [image_array]  
        })
        andmestik_df = pd.concat([andmestik_df,image_data], ignore_index=True)


# Kui soovite tulemused salvestada uude CSV faili
andmestik_df.to_csv('reshaped_file.csv', index=True)
# Vaatame ümberstruktureeritud andmeid
print(andmestik_df.info())

KeyboardInterrupt: 

In [20]:
# Load the data
#df = pd.read_csv('reshaped_file.csv')  # assuming your data is in a CSV file
df = andmestik_df

# Drop rows with missing ratings
df = df.dropna(subset=['usaldusväärsus', 'ilu', 'intelligents'])

# Extract the images and labels
images = np.stack(df['pilt'].values)
ratings = df[['usaldusväärsus', 'ilu', 'intelligents']].values

# Normalize image data
images = images / 255.0

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(images, ratings, test_size=0.2, random_state=42)

# Define the CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(600, 600, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(3)  # Output layer for the 3 ratings
])

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

# Train the model
history = model.fit(X_train, y_train, epochs=25, batch_size=16, validation_split=0.2)

# Evaluate the model
loss, mae = model.evaluate(X_test, y_test)
print(f'Model evaluation - Loss: {loss}, MAE: {mae}')

# Save the model
model.save('face_ratings_model.h5')

C:\Users\kost\AppData\Local\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/25
1/1 ━━━━━━━━━━━━━━━━━━━━ 10s 10s/step - loss: 43.8635 - mae: 6.1694 - val_loss: 12148.9482 - val_mae: 109.7381
Epoch 2/25
1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step - loss: 12027.5801 - mae: 105.8244 - val_loss: 57.9376 - val_mae: 6.0829
Epoch 3/25
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step - loss: 73.7829 - mae: 6.3102 - val_loss: 125.3441 - val_mae: 10.5204
Epoch 4/25
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step - loss: 166.5204 - mae: 12.0979 - val_loss: 75.2971 - val_mae: 8.1841
Epoch 5/25
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step - loss: 111.6534 - mae: 10.0324 - val_loss: 43.3864 - val_mae: 6.1376
Epoch 6/25
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step - loss: 66.7333 - mae: 7.6287 - val_loss: 27.6405 - val_mae: 4.7568
Epoch 7/25
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step - loss: 44.3665 - mae: 6.1244 - val_loss: 19.7355 - val_mae: 3.8580
Epoch 8/25
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step - loss: 29.4694 - mae: 4.8302 - val_loss: 13.6826 - val_mae: 3.0322
Epoch 9/25
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step - loss: 24.5008

Model evaluation - Loss: 5.553903102874756, MAE: 1.9667624235153198


In [29]:
# Load the trained model
model = load_model('face_ratings_model.h5')

# Function to preprocess the image
def preprocess_image(image_path):
    image = Image.open(image_path)
    
    image = image.crop(coords_for_crop(image_path))
    image.show()
    image = image.resize((600, 600))  # Ensure the image is 600x600 pixels
    image = np.array(image) / 255.0  # Normalize the image
    image = np.expand_dims(image, axis=0)  # Add batch dimension
    return image

# Path to the random image
random_image_path = 'people/brad.jpg'  # Replace with the actual path

# Preprocess the image
preprocessed_image = preprocess_image(random_image_path)

# Predict the ratings
predicted_ratings = model.predict(preprocessed_image)

# Print the predicted ratings
usaldusvaarus, ilu, intelligents = predicted_ratings[0]
print(f"Predicted Reliability (usaldusväärsus): {usaldusvaarus}")
print(f"Predicted Attractiveness (ilu): {ilu}")
print(f"Predicted Intelligence (intelligents): {intelligents}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 212ms/step
Predicted Reliability (usaldusväärsus): 2.830345630645752
Predicted Attractiveness (ilu): 3.0858256816864014
Predicted Intelligence (intelligents): 3.032499313354492
